### Download all the necessary libraries

In [21]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import math
import json # library to handle JSON files
print ("next install conda-forge geopy")
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Libraries to read HTML files
from lxml import html

print ("next install conda-forge folium=0.5.0")
# Version not working
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

! pip install geocoder
import geocoder # import geocoder


import folium # map rendering library

print('Libraries imported.')


next install conda-forge geopy
Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt

next install conda-forge folium=0.5.0
Solving environment: | ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


## 1. Download and Scrape Toronto neighbourhoods data set from the Web

Using the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [22]:
# Get the boroughs from ZIP's table on wiki site
toronto_pages = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tzips = pd.DataFrame(toronto_pages[0])
print('Original Data Set: ')
print(tzips.shape)
print(tzips.head(30))
print(tzips.tail())

Original Data Set: 
(287, 3)
   Postcode           Borough     Neighbourhood
0       M1A      Not assigned      Not assigned
1       M2A      Not assigned      Not assigned
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M6A        North York  Lawrence Heights
6       M6A        North York    Lawrence Manor
7       M7A  Downtown Toronto      Queen's Park
8       M8A      Not assigned      Not assigned
9       M9A         Etobicoke  Islington Avenue
10      M1B       Scarborough             Rouge
11      M1B       Scarborough           Malvern
12      M2B      Not assigned      Not assigned
13      M3B        North York   Don Mills North
14      M4B         East York  Woodbine Gardens
15      M4B         East York     Parkview Hill
16      M5B  Downtown Toronto           Ryerson
17      M5B  Downtown Toronto   Garden District
18      M6B        North York         Glencairn
19      M7B

### 1.1 Data wrangling

In [23]:
# 2.1 Ignoring Cells with Borough Not Assigned
tzips_wrg = tzips[tzips['Borough'] != 'Not assigned']
tzips_wrg = tzips_wrg.sort_values(['Postcode'], ascending = (True))
tzips_wrg = tzips_wrg.reset_index(drop=True)
print('Data Set with Borough cleaned')
print('Shape: ')
print(tzips_wrg.shape)
print('Head: ')
print(tzips_wrg.head(10))
print('Tail: ')
print(tzips_wrg.tail(10))

# 2.2 Checking neighbourhood not assigned

for post_x in range(0,len(tzips_wrg)):

    keynbh = tzips_wrg.loc[post_x, 'Neighbourhood']
    
    if keynbh  == '':
        print (post_x, ' ,No Neigh..')
        tzips_wrg.loc[post_x,'Neighbourhood']=tzips_wrg.loc[post_x,'Borough']
          
# 2.3 Merging equal Post code
# define the new dataframe columns
column_names = ['Postcode', 'Borough', 'Neighbourhood'] 
# instantiate the dataframe (empty only with column names)
tzips_wrg1 = pd.DataFrame(columns=column_names)
tzips_wrg = tzips_wrg.sort_values(['Postcode'], ascending = (True))
Mrgn=0
post_xpr = 0
for post_x in range(0,len(tzips_wrg)):

    keywrg = tzips_wrg.loc[post_x, 'Postcode']
    
    if keywrg  != post_xpr:
        tzips_wrg1 = tzips_wrg1.append(tzips_wrg.loc[post_x])
        mrgn=0
    else:
        mrgn= mrgn + 1
        tzips_wrg1.loc[post_x-mrgn,'Neighbourhood']=tzips_wrg1.loc[post_x-mrgn,'Neighbourhood']+', '+tzips_wrg.loc[post_x, 'Neighbourhood']
        
    post_xpr = keywrg   

Data Set with Borough cleaned
Shape: 
(210, 3)
Head: 
  Postcode      Borough   Neighbourhood
0      M1B  Scarborough           Rouge
1      M1B  Scarborough         Malvern
2      M1C  Scarborough      Port Union
3      M1C  Scarborough      Rouge Hill
4      M1C  Scarborough  Highland Creek
5      M1E  Scarborough       Guildwood
6      M1E  Scarborough     Morningside
7      M1E  Scarborough       West Hill
8      M1G  Scarborough          Woburn
9      M1H  Scarborough       Cedarbrae
Tail: 
    Postcode    Borough         Neighbourhood
200      M9R  Etobicoke  Martin Grove Gardens
201      M9V  Etobicoke        Albion Gardens
202      M9V  Etobicoke      Beaumond Heights
203      M9V  Etobicoke            Humbergate
204      M9V  Etobicoke             Jamestown
205      M9V  Etobicoke           Mount Olive
206      M9V  Etobicoke         South Steeles
207      M9V  Etobicoke           Thistletown
208      M9V  Etobicoke           Silverstone
209      M9W  Etobicoke             Nor

### 1.2. Wrangled Data set.  Number of rows, head and tail information (tzips_wrg1)

In [24]:
tzips_wrg1 = tzips_wrg1.sort_values(['Postcode'], ascending = (True))
tzips_wrg1 = tzips_wrg1.reset_index(drop=True)
print('')
print('Wrangled Data Set')
print('Shape: ')
print(tzips_wrg1.shape)
print('Head: ')
print(tzips_wrg1.head(30))
print('Tail: ')
print(tzips_wrg1.tail(10))




Wrangled Data Set
Shape: 
(103, 3)
Head: 
   Postcode      Borough                                      Neighbourhood
0       M1B  Scarborough                                     Rouge, Malvern
1       M1C  Scarborough             Port Union, Rouge Hill, Highland Creek
2       M1E  Scarborough                  Guildwood, Morningside, West Hill
3       M1G  Scarborough                                             Woburn
4       M1H  Scarborough                                          Cedarbrae
5       M1J  Scarborough                                Scarborough Village
6       M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park
7       M1L  Scarborough                    Golden Mile, Oakridge, Clairlea
8       M1M  Scarborough    Cliffcrest, Scarborough Village West, Cliffside
9       M1N  Scarborough                        Cliffside West, Birch Cliff
10      M1P  Scarborough  Wexford Heights, Dorset Park, Scarborough Town...
11      M1R  Scarborough                     

##  2. Get Latitude and Longitude of the Boroughs postal codes

###  2.1 Trying geolocator.geocode. However it finishes with "service not available" after some iterations. I have decided to use the CSV file.

In [25]:
tzips_wrg1 = tzips_wrg1.reset_index(drop=True)

for nx in range(0,len(tzips_wrg1)):

    post_c = tzips_wrg1.loc[nx, 'Postcode']
          
# initialize your variable to None
    lat_lng_coords = None
    iterx = 0

# loop until you get the coordinates
    while(lat_lng_coords is None):
        address = '{}, Toronto, Ontario'.format(post_c)
        g = geolocator.geocode(address)
        iterx = iterx + 1        
        if iterx >= 3:
           lat_lng_coords =[0,0]
       
    
    tzips_wrg1.loc[nx,'latitude']= lat_lng_coords[0]
    tzips_wrg1.loc[nx,'longitude']= lat_lng_coords[1]
    print(nx)
    
tzips_wrg1.shape

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


GeocoderUnavailable: Service not available

###  2.2 Since "geolocator.geocode" is not responding properly, I will use the CSV file instead

In [26]:
# Read CSV data
to_coordx = pd.read_csv('http://cocl.us/Geospatial_data')
to_coord = pd.DataFrame(to_coordx).set_index('Postal Code')
to_coord.head(10)


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


###  2.3 Adding Latitude & Longitude to then data set (tzipz_wrg2)

In [27]:
tzips_wrg2=tzips_wrg1
tzips_wrg2['Latitude']=np.nan
tzips_wrg2['Longitude']=np.nan
tzips_wrg2.head()
for nx in range(0,len(tzips_wrg2)):
    pcode = tzips_wrg2.loc[nx,'Postcode']
    tzips_wrg2.loc[nx, 'Latitude']=to_coord.loc[pcode,'Latitude']
    tzips_wrg2.loc[nx, 'Longitude']=to_coord.loc[pcode,'Longitude']

    



In [3]:
###  2.4 Printing the data set and the Map of neighborhoods of Toronto

In [28]:
print(tzips_wrg2.shape)
tzips_wrg2.head(12)


(103, 7)


,Postcode,Borough,Neighbourhood,latitude,longitude,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",0.0,0.0,43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",0.0,0.0,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.0,0.0,43.763573,-79.188711
3,M1G,Scarborough,Woburn,0.0,0.0,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,0.0,0.0,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,0.0,0.0,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.0,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",0.0,0.0,43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",0.0,0.0,43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",0.0,0.0,43.692657,-79.264848


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [29]:

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latto = location.latitude
lonto = location.longitude
print('The geograpical coordinates of Toronto City are {}, {}.'.format(latto, lonto))

The geograpical coordinates of Toronto City are 43.653963, -79.387207.


In [161]:
#! conda install -c conda-forge folium
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latto, lonto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tzips_wrg2['Latitude'], tzips_wrg2['Longitude'], tzips_wrg2['Borough'], tzips_wrg2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [32]:
#### Define Foursquare Credentials and Version
CLIENT_ID = 'TMIKXOISICLWJJQODBQEWYOC44TQ50DL223HISL2NP0BGO14' # your Foursquare ID
CLIENT_SECRET = 'G1IDMYTRAKU1OP4RW1S35YJJ4OEDYX2UIR44UI3QEJZHNLQY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TMIKXOISICLWJJQODBQEWYOC44TQ50DL223HISL2NP0BGO14
CLIENT_SECRET:G1IDMYTRAKU1OP4RW1S35YJJ4OEDYX2UIR44UI3QEJZHNLQY


In [33]:
#### Let's explore the first neighborhood in our dataframe.
print (tzips_wrg2.loc[0, 'Neighbourhood'])

neigh_lat = tzips_wrg2.loc[0, 'Latitude'] # neighborhood latitude value
neigh_lon = tzips_wrg2.loc[0, 'Longitude'] # neighborhood longitude value

neigh_name = tzips_wrg2.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_lon))

Rouge, Malvern
Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [34]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lon, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TMIKXOISICLWJJQODBQEWYOC44TQ50DL223HISL2NP0BGO14&client_secret=G1IDMYTRAKU1OP4RW1S35YJJ4OEDYX2UIR44UI3QEJZHNLQY&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [35]:
# Send the GET request and examine the resutls
import json # library to handle JSON files
results = requests.get(url).json()
# results  # too much data
## How is the newyork_data structured?
print ("results structure Dictionary Keys:  ", results.keys())

results structure Dictionary Keys:   dict_keys(['meta', 'response'])


In [36]:
print("response structure Dictionary Keys:  ", results['response'].keys())

response structure Dictionary Keys:   dict_keys(['warning', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])


In [37]:
print("groups structure Dictionary Keys:  ", results['response']['groups'])

groups structure Dictionary Keys:   [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bb6b9446edc76b0d771311c', 'name': "Wendy's", 'location': {'crossStreet': 'Morningside & Sheppard', 'lat': 43.80744841934756, 'lng': -79.19905558052072, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80744841934756, 'lng': -79.19905558052072}], 'distance': 387, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d16e941735', 'name': 'Fast Food Restaurant', 'pluralName': 'Fast Food Restaurants', 'shortName': 'Fast Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4bb6b9446edc76b0d771311c-0'}]}]


In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
# This line added to understand venues structure
#import json # library to handle JSON files

venues = results['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues) # flatten JSON copy to print below
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# Now we are ready to clean the json and structure it into a pandas dataframe.
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)
print(nearby_venues.head())

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

(1, 4)
      name            categories        lat        lng
0  Wendy's  Fast Food Restaurant  43.807448 -79.199056
1 venues were returned by Foursquare.


In [41]:
# This line added to understand venues structure
print (venues)


[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bb6b9446edc76b0d771311c', 'name': "Wendy's", 'location': {'crossStreet': 'Morningside & Sheppard', 'lat': 43.80744841934756, 'lng': -79.19905558052072, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80744841934756, 'lng': -79.19905558052072}], 'distance': 387, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d16e941735', 'name': 'Fast Food Restaurant', 'pluralName': 'Fast Food Restaurants', 'shortName': 'Fast Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4bb6b9446edc76b0d771311c-0'}]


In [42]:
# This line added to understand nearby_venues structure
print(nearby_venues1)

   reasons.count                                      reasons.items  \
0              0  [{'summary': 'This spot is popular', 'type': '...   

                       referralId  \
0  e-0-4bb6b9446edc76b0d771311c-0   

                                    venue.categories  \
0  [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...   

                   venue.id venue.location.cc venue.location.city  \
0  4bb6b9446edc76b0d771311c                CA             Toronto   

  venue.location.country venue.location.crossStreet  venue.location.distance  \
0                 Canada     Morningside & Sheppard                      387   

  venue.location.formattedAddress  \
0            [Toronto ON, Canada]   

                       venue.location.labeledLatLngs  venue.location.lat  \
0  [{'label': 'display', 'lat': 43.80744841934756...           43.807448   

   venue.location.lng venue.location.state venue.name  venue.photos.count  \
0          -79.199056                   ON    Wendy's           

## 3. Explore Neighborhoods in Toronto

In [43]:
# Function to repeat the same process to all the neighbourhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
# Code to run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.
to_venues = getNearbyVenues(names=tzips_wrg2['Neighbourhood'],
                                   latitudes=tzips_wrg2['Latitude'],
                                   longitudes=tzips_wrg2['Longitude']
                                  )

# Let's check the size of the resulting dataframe
print(to_venues.shape)
print(to_venues.head())


# Let's check how many venues were returned for each neighborhood
print(tzips_wrg2.groupby('Neighbourhood').count())

(2228, 7)
                             Neighborhood  Neighborhood Latitude  \
0                          Rouge, Malvern              43.806686   
1  Port Union, Rouge Hill, Highland Creek              43.784535   
2  Port Union, Rouge Hill, Highland Creek              43.784535   
3       Guildwood, Morningside, West Hill              43.763573   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                   Venue  Venue Latitude  \
0              -79.194353                 Wendy's       43.807448   
1              -79.160497  Chris Effects Painting       43.784343   
2              -79.160497   Royal Canadian Legion       43.782533   
3              -79.188711       G & G Electronics       43.765309   
4              -79.188711              Marina Spa       43.766000   

   Venue Longitude              Venue Category  
0       -79.199056        Fast Food Restaurant  
1       -79.163742  Construction & Landscaping  
2       -79.163085 

In [28]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(to_venues['Venue Category'].unique())))

There are 267 uniques categories.


### 3.1 Analyze Each Neighborhood

In [45]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighbourhood'] = to_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [47]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
to_grouped = to_onehot.groupby('Neighbourhood').mean().reset_index()

print(to_grouped.shape)
print(to_grouped.head(15))


(99, 269)
                                        Neighbourhood  Accessories Store  \
0                            Adelaide, Richmond, King                0.0   
1                                           Agincourt                0.0   
2   Albion Gardens, Beaumond Heights, Humbergate, ...                0.0   
3     Bathurst Manor, Wilson Heights, Downsview North                0.0   
4                                     Bayview Village                0.0   
5                   Bedford Park, Lawrence Manor East                0.0   
6                                         Berczy Park                0.0   
7   Business Reply Mail Processing Centre 969 Eastern                0.0   
8                                 Caledonia-Fairbanks                0.0   
9               Canada Post Gateway Processing Centre                0.0   
10                                          Cedarbrae                0.0   
11                                 Central Bay Street                0.0   
12

#### Let's print each neighborhood along with the top 5 most common venues

In [48]:
num_top_venues = 5

for hood in to_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, Richmond, King----
             venue  freq
0      Coffee Shop  0.07
1       Restaurant  0.05
2             Café  0.04
3  Thai Restaurant  0.04
4              Bar  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1               Skating Rink  0.25
2  Latin American Restaurant  0.25
3             Breakfast Spot  0.25
4              Metro Station  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, South Steeles, Thistletown, Silverstone----
                  venue  freq
0         Grocery Store  0.18
1   Fried Chicken Joint  0.09
2        Sandwich Place  0.09
3  Fast Food Restaurant  0.09
4              Pharmacy  0.09


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop  0.11
1        Shopping Mall  0.05
2  Fried Chicken Joint  0.05
3   Frozen Yogurt Shop  0.05
4     Sushi Restaurant  0.05


----Bayview Village----
                 venue  freq


#### Let's put that into a *pandas* dataframe

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [138]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neigh_venues_sorted = pd.DataFrame(columns=columns)
neigh_venues_sorted['Neighbourhood'] = to_grouped['Neighbourhood']

for ind in np.arange(to_grouped.shape[0]):
    neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neigh_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Restaurant,Thai Restaurant,Café,Sushi Restaurant,Steakhouse,Bar,Seafood Restaurant,Bakery,Gym
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Video Store,Fried Chicken Joint,Discount Store,Sandwich Place,Japanese Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Beer Store
3,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Frozen Yogurt Shop,Restaurant,Supermarket,Ice Cream Shop,Sushi Restaurant,Deli / Bodega,Middle Eastern Restaurant,Pizza Place,Pharmacy
4,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
5,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Sushi Restaurant,Pharmacy,Juice Bar,Liquor Store,Indian Restaurant,Fast Food Restaurant
6,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Beer Bar,Café,Farmers Market,Cheese Shop,Bistro
7,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Moving Target,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery
8,Caledonia-Fairbanks,Park,Market,Women's Store,Airport,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
9,Canada Post Gateway Processing Centre,Intersection,Hotel,Coffee Shop,Middle Eastern Restaurant,Gym,Fried Chicken Joint,Sandwich Place,American Restaurant,Mediterranean Restaurant,Burrito Place


## 4. Cluster Neighborhoods

### 4.1 Run k-means to cluster the neighborhood into 5 clusters.

In [139]:
# set number of clusters
kclusters = 3

to_grouped_clustering =to_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print (len(kmeans.labels_))
kmeans.labels_[0:] 

99


array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

### 4.1 Add Clustering labels to the DF

In [140]:
# add clustering labels
neigh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = tzips_wrg2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(neigh_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

to_merged.head(16) # check the last columns!

,Postcode,Borough,Neighbourhood,latitude,longitude,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",0.0,0.0,43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",0.0,0.0,43.784535,-79.160497,2.0,Bar,Construction & Landscaping,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.0,0.0,43.763573,-79.188711,2.0,Rental Car Location,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Bank,Electronics Store,Intersection,Dim Sum Restaurant,Diner
3,M1G,Scarborough,Woburn,0.0,0.0,43.770992,-79.216917,2.0,Coffee Shop,Convenience Store,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,0.0,0.0,43.773136,-79.239476,2.0,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Bank,Hakka Restaurant,Donut Shop,Doner Restaurant
5,M1J,Scarborough,Scarborough Village,0.0,0.0,43.744734,-79.239476,2.0,Playground,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",0.0,0.0,43.727929,-79.262029,2.0,Discount Store,Department Store,Hobby Shop,Coffee Shop,Bus Station,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Yoga Studio
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",0.0,0.0,43.711112,-79.284577,2.0,Bakery,Park,Ice Cream Shop,Soccer Field,Metro Station,Bus Line,Intersection,Yoga Studio,Discount Store,Diner
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",0.0,0.0,43.716316,-79.239476,2.0,American Restaurant,Motel,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
9,M1N,Scarborough,"Cliffside West, Birch Cliff",0.0,0.0,43.692657,-79.264848,2.0,Skating Rink,Café,College Stadium,General Entertainment,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


### 4.2 Let's visualize the resulting clusters in Toronto Map

In [152]:
# create map
map_clusters = folium.Map(location=[latto, lonto], zoom_start=11)

# set color scheme for the clusters
colorsx=['Red', '#007849', 'Blue', 'Brown']
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    if cluster in range(0, 5):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=3,
            popup=label,
            color=colorsx[int(cluster)],
            fill=True,
            fill_color=colorsx[int(cluster)],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4.3 Examine Clusters

In [155]:
# Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,43.815252,-79.284577,0.0,Park,Coffee Shop,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
20,North York,43.757490,-79.374714,0.0,Park,Cafeteria,College Rec Center,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
23,North York,43.752758,-79.400049,0.0,Park,Convenience Store,Bank,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Department Store
25,North York,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
30,North York,43.737473,-79.464763,0.0,Park,Bus Stop,Airport,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
31,North York,43.739015,-79.506944,0.0,Grocery Store,Bank,Hotel,Park,Airport Food Court,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
40,East York,43.685347,-79.338106,0.0,Park,Coffee Shop,Convenience Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
44,Central Toronto,43.728020,-79.388790,0.0,Park,Swim School,Bus Line,Construction & Landscaping,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Department Store,Dog Run
50,Downtown Toronto,43.679563,-79.377529,0.0,Park,Playground,Trail,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
64,Central Toronto,43.696948,-79.411307,0.0,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [156]:
# Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 1, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,43.636258,-79.498509,1.0,Baseball Field,Construction & Landscaping,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
97,North York,43.724766,-79.532242,1.0,Baseball Field,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant


In [157]:
# Cluster 3
to_merged.loc[to_merged['Cluster Labels'] == 2, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
1,Scarborough,43.784535,-79.160497,2.0,Bar,Construction & Landscaping,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
2,Scarborough,43.763573,-79.188711,2.0,Rental Car Location,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Bank,Electronics Store,Intersection,Dim Sum Restaurant,Diner
3,Scarborough,43.770992,-79.216917,2.0,Coffee Shop,Convenience Store,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,Scarborough,43.773136,-79.239476,2.0,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Bank,Hakka Restaurant,Donut Shop,Doner Restaurant
5,Scarborough,43.744734,-79.239476,2.0,Playground,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
6,Scarborough,43.727929,-79.262029,2.0,Discount Store,Department Store,Hobby Shop,Coffee Shop,Bus Station,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Yoga Studio
7,Scarborough,43.711112,-79.284577,2.0,Bakery,Park,Ice Cream Shop,Soccer Field,Metro Station,Bus Line,Intersection,Yoga Studio,Discount Store,Diner
8,Scarborough,43.716316,-79.239476,2.0,American Restaurant,Motel,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
9,Scarborough,43.692657,-79.264848,2.0,Skating Rink,Café,College Stadium,General Entertainment,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [158]:
# Cluster 4
to_merged.loc[to_merged['Cluster Labels'] == 3, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [159]:
# Cluster 5
to_merged.loc[to_merged['Cluster Labels'] == 4, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
